# RIST Internship Test - Nichita Uțiu
In this document we will try to train a model to predict the type of a sorting algorithm based on a visual representation of it over 42 iterations on a vector of 10 elements(each iteration being a swap).

In [2]:
%matplotlib inline
# import needed libraries
# pandas
import pandas as pd

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# this styling is purely my preference
# less chartjunk
sns.set_context('notebook', font_scale=1.5, rc={'line.linewidth': 2.5})
sns.set(style='ticks', palette='Set2')

In [3]:
algorithms_df = pd.read_csv('./data.csv')  # load the data from  the csv
algorithms_df.describe()  # view a short summary

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel410,pixel411,pixel412,pixel413,pixel414,pixel415,pixel416,pixel417,pixel418,pixel419
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4.500000,0.492175,0.498129,0.512111,0.500150,0.497343,0.503398,0.498808,0.489210,0.510043,...,0.499388,0.500035,0.500087,0.498844,0.497845,0.497845,0.498844,0.500087,0.500035,0.499388
std,2.872425,0.293948,0.290994,0.294248,0.290129,0.280255,0.288480,0.290329,0.290039,0.285080,...,0.416720,0.337148,0.261106,0.194865,0.151147,0.151147,0.194865,0.261106,0.337148,0.416720
min,0.000000,0.000400,0.000500,0.000300,0.001200,0.000300,0.000000,0.000100,0.000400,0.001800,...,0.000000,0.002900,0.015400,0.062500,0.085000,0.085000,0.062500,0.015400,0.002900,0.000000
25%,2.000000,0.232000,0.253100,0.253100,0.253975,0.259150,0.254000,0.237625,0.243000,0.268175,...,0.065600,0.163250,0.256525,0.341550,0.392675,0.392675,0.341550,0.256525,0.163250,0.065600
50%,4.500000,0.491900,0.490000,0.512050,0.494800,0.496650,0.512950,0.500700,0.483250,0.513550,...,0.458600,0.486250,0.506850,0.502200,0.497650,0.497650,0.502200,0.506850,0.486250,0.458600
75%,7.000000,0.751150,0.750525,0.771700,0.752225,0.735500,0.749125,0.746925,0.732625,0.750600,...,0.933675,0.842000,0.738300,0.650725,0.607250,0.607250,0.650725,0.738300,0.842000,0.933675
max,9.000000,1.000000,0.997700,0.999400,0.999800,0.996100,0.997400,0.998200,0.999300,0.999500,...,1.000000,0.993900,0.987100,0.942800,0.890600,0.890600,0.942800,0.987100,0.993900,1.000000


## Splitting and looking at the data

In [4]:
Y = algorithms_df['label'].astype(int)  # the labels
X = algorithms_df.drop('label', axis=1)  # everyhing but the labels

# count by label
print("Number of labels = {}".format(Y.unique().size))

# iterate over the label,count pairs
# use sort_index as value_counts returns them unordered
for ind, cnt in Y.value_counts().sort_index().iteritems():
    print("Samples with label {}: {}".format(ind, cnt))

Number of labels = 10
Samples with label 0: 1000
Samples with label 1: 1000
Samples with label 2: 1000
Samples with label 3: 1000
Samples with label 4: 1000
Samples with label 5: 1000
Samples with label 6: 1000
Samples with label 7: 1000
Samples with label 8: 1000
Samples with label 9: 1000


## Sample inspection

In [5]:
def get_order(arr):
    """Returns whether the elements of the given numpy
    array are ascending, descending or unordered.
    
    :return `-1` if descending, `1` if ascending, `0` if
    ordered
    """
    
    # compare elements pairwise
    # using numpy for vectorisation
    if np.all(arr[:-1] <= arr[1:]):
        return 1 # ascending 
    if np.all(arr[:-1] >= arr[1:]):
        return -1 # descending 
    return 0

def get_samp_order(samp, width=10):
    """Recives a sample and returns it's last row's
    order. Receives the width of the rows, considering
    the image is unrolled(defualt 10)."""
    return get_order(samp[-10:])

Exmaple:

In [6]:
SAMPLE_IND = 10
IMG_WIDTH = 10

# get the sample
sample = X.iloc[SAMPLE_IND].values

order = get_samp_order(sample)
print("Sample {} order: {} ".format(SAMPLE_IND, order))
print("Last row = {}".format(sample[-IMG_WIDTH:]))  # get the last row
# TODO: IMAGE

Sample 10 order: 1 
Last row = [ 0.0092  0.0178  0.3939  0.4125  0.4881  0.5074  0.6198  0.7261  0.7501
  0.9777]


## Feature extraction

In [15]:
def extract_ftds(samps, width=10):
    """Extracts for each sample, the first row where
    each each element changes its initial position.
    
    :returns a matrix of where each row is the features
    of each sample
    """
    
    # define the function for a single row
    def extract_ftd(samp):
        img = samp.reshape((-1, width)) # reverse into a matrix
        # check what elements on each row are dfferent to the first
        # that means that the first True value is the first swap
        changes = img != img[0]
        # return the row index of the change
        return np.argmax(changes, axis=0)
    
    # vectorize it
    return np.apply_along_axis(extract_ftd, 1, samps)
 
    
def extract_ftps(samps, width=10):
    """Extracts for each sample, the first row where
    each element reaches it's final position.
    
    :returns a matrix of where each row is the features
    of each sample"""

    # same as above
    def extract_ftp(samp):
        img = samp.reshape((-1, width))
        # however this time check them in reverse
        changes = (img != img[-1])[::-1, :]
        # return the row index of the change
        # substract from len to get the actual postion
        return changes.shape[0] - np.argmax(changes, axis=0)

    return np.apply_along_axis(extract_ftp, 1, samps)

def extract_rpas(samps, width=10):
    """Extracts for each sample, the area of image
    where the algorithm is active.
    
    :returns a column vector of the areas for each
    corresponding row"""
    
    # use the ftp features, sum them to get the inactive
    # area and substract it from the total
    # note: the vectorized computation is fast enough
    # not to warrant precalculating ftp
    inactive_area = extract_ftps(samps, width).sum(axis=1)
    
    # substract them from the total size of an image
    # also add a dimension to convert it to a matrix
    return (samps[0].size - inactive_area)[:, np.newaxis]
    

### Bonus -- Additional Features
1) We can compute for each element of the array,(given its initial value) the maximum number of consecutive swaps. This feature could differentiate between algorithms which
2) We can add a one-hot encoded feature from the order of the last row. This one would be linearly correlated to whether the algorithm is ascending or descending. It could also tell apart slower algorithms which may not finish in the 42 given steps.
3) 

In [16]:
X.values.sum(axis=1)[:, np.newaxis]

array([[ 221.9322],
       [ 221.9322],
       [ 221.9322],
       ..., 
       [ 212.9862],
       [ 212.9862],
       [ 212.9862]])